<a href="https://colab.research.google.com/github/buganart/stylegan2-ada-pytorch/blob/main/stylegan2ada_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

# Description


In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
#@markdown - The database directory
database_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/Google_Street_View" #@param {type:"string"}
# experiment_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan" #@param {type:"string"}
#@markdown - The output directory of experiment data 
out_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-stylegan2" #@param {type:"string"}
#@markdown - set wandb run id of logged run to resume from there
resume_id = "" #@param {type:"string"}
#@markdown - If the data is still not being processed, this defines the size of processed image.
resolution = "1024" #@param {type:"string"}

In [ ]:
#@title Clone stylegan2 repo

!git clone https://github.com/buganart/stylegan2-ada-pytorch

In [ ]:
#@title Install Dependencies

%cd /content/stylegan2-ada-pytorch
# %pip install -r requirements.txt
%pip install wandb
%pip install ninja
%pip install pyspng
clear_on_success("Dependencies installed.")

In [ ]:
#@title pre-process data 
print(database_dir)
database_dir = Path(database_dir)
out_dir = Path(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)

if database_dir.is_file() and database_dir.suffix == ".zip":
    datazip_dir = database_dir
elif database_dir.is_dir():
    zipfile = sorted(database_dir.rglob(f"*_{resolution}.zip"))
    # pick any zipfile in the directory
    if len(zipfile) > 0 :
        datazip_dir = zipfile[0]
    else:
        print(database_dir)
        # process data
        unpack_data = Path("/tmp/unpack")
        unpack_data.mkdir(parents=True, exist_ok=True)
        # copy files to tmp dir for processing
        !find "{database_dir}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$unpack_data"/

        # process data
        target_zipfile = database_dir / f"data_{resolution}.zip"
        !python dataset_tool.py --source="{unpack_data}" --dest="{target_zipfile}" --width="{resolution}" --height="{resolution}"
        datazip_dir = target_zipfile
else:
    raise Exception("database_dir is invalid.")
# clear_on_success("All files copied to this runtime.")


In [ ]:
#@title TRAIN
# os.environ["WANDB_MODE"] = "dryrun"

# unzip files to local tmp folder
unzip_dataset_dir = Path("/tmp/data")
unzip_dataset_dir.mkdir(parents=True, exist_ok=True)
!unzip "{datazip_dir}" -d "{unzip_dataset_dir}"


!python train.py \
--outdir "$out_dir" \
--data "$unzip_dataset_dir" \
--resume "$resume_id"